In [1]:
import time
import os

import pandas as pd
import numpy as np

import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
#os.chdir('/c/Users/matth/Documents/Coding/RA Fouirnaies')

In [3]:
#!pwd

# Vote Smart - FollowTheMoney Link Table

## Overview

This notebook documents the process of creating a link table between the enterprise/organization ID of Vote Smart and FollowTheMoney. We begin with a list of the desired organizations' names as seen on the Vote Smart website and scrape the FTM website search queries to obtain unique IDs. 

### Search Query Exploration

#### Example: Campaign for Working Families

Here we seek to understand the format of the search query output and create a method to extract the desired information from each query while handling/providing discussion of how to handle exceptions.

In [4]:
url = 'https://www.followthemoney.org/search-results/SearchForm?Search=Campaign+for+Working+Families'
headers = {'Access-Control-Allow-Origin': '*',
          'Access-Control-Allow-Methods': 'GET',
          'Access-Control-Allow-Headers': 'Content-Type',
          'Access-Control-Max-Age': '3600'}

**Issue**

Direct parsing of the html using beautifulsoup does not work since the table produced by the search query is a dynamic table, and as a result you need to actually make a request to the website for it to show up.

Debugging for dynamic table issue

https://stackoverflow.com/questions/17597424/how-to-retrieve-the-values-of-dynamic-html-content-using-python
https://stackoverflow.com/questions/40208051/selenium-using-python-geckodriver-executable-needs-to-be-in-path

https://www.howtogeek.com/118594/how-to-edit-your-system-path-for-easy-command-line-access/amp/

**Workaround**

Use selenium webdriver (requires firefox and geckodriver) to access the table. This is not ideal since it requires certain downloads/specific file paths in addition to opening the firefox broswer. Based on what I have read, it appears that this is the easiest way to get around the issue of a dynamic table. As a result, we will be using this method to create the link table. I will update the file paths to be generic and add the necessary files to the main directory so that this can be run elsewhere. 

An additional 

In [5]:
#driver = webdriver.Firefox() #executable_path=r'your\path\geckodriver.exe'
#driver.get(url)

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

start_time = time.time()


options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\matth\Documents\Coding\RA Fouirnaies\geckodriver.exe', options=options)
driver.get(url)

html = driver.page_source

driver.close()

print("My program took", time.time()-start_time, "to run")

My program took 10.939863204956055 to run


In [6]:
start_time = time.time()

#req = requests.get(html, headers)
sample = BeautifulSoup(html, 'html.parser') #req.content

print("My program took", time.time()-start_time, "to run")

My program took 0.07384371757507324 to run


In [7]:
# navigating to dynamic search query table and obtaining a list of the 'Name' element/col of each row
query_table = sample.find_all(name='div', class_ = 'table-responsive')
query_list = query_table[0].find_all(name='td', style = 'text-align: left;')
len(query_list)

3

In [8]:
# creating mock link table

names = []
ids = []

for row in query_list:
    names.append(row.string)
    ids.append(row['tokenvalue'])
    print(row.string + ': ' + row['tokenvalue'])
    
dict = {'org_name': names, 'ID': ids}

df = pd.DataFrame(dict)
df

CAMPAIGN FOR WORKING FAMILIES: 471
CAMPAIGN FOR WORKING FAMILIES PAC CWF: 46277079
NEW HAMPSHIRE CAMPAIGN FOR WORKING FAMILIES: 16629025


,org_name,ID
0,CAMPAIGN FOR WORKING FAMILIES,471
1,CAMPAIGN FOR WORKING FAMILIES PAC CWF,46277079
2,NEW HAMPSHIRE CAMPAIGN FOR WORKING FAMILIES,16629025


### Search & Match Automation

#### Query Creation

For both tasks we assume that we are given a list of strings where each string is an organization name that is separated by spaces (if longer than one word) and is of unknown capitalization. 

In [9]:
in_string = 'campaign for working families' # representative string in the list of organizations




#### Organization Matching